### 使用 wkhtmltopdf 将 HTML 转换为 PDF
#### 期货数据中包含PDF和HTML两种格式文件，需要将HTML文件转换为PDF文件

In [11]:
# 配置项
DATA_DIR = "../data/20250401"
OUTPUT_DIR = "../output"

import os
from pdfminer.high_level import extract_text

def extract_text_to_txt(pdf_path, txt_dir):
    """将PDF文件内容提取到TXT文件中"""
    try:
        # 创建TXT文件名（与PDF同名）
        txt_filename = os.path.splitext(os.path.basename(pdf_path))[0] + ".txt"
        txt_path = os.path.join(txt_dir, txt_filename)
        
        # 提取PDF文本
        text = extract_text(pdf_path)
        
        # 写入TXT文件
        with open(txt_path, 'w', encoding='utf-8') as f:
            f.write(text)
            
        return True
    except Exception as e:
        print(f"处理文件 {pdf_path} 时出错: {str(e)}")
        return False

def main():
    os.makedirs(OUTPUT_DIR, exist_ok=True)
    txt_output_dir = os.path.join(OUTPUT_DIR, "text_files")
    os.makedirs(txt_output_dir, exist_ok=True)

    # 获取所有PDF文件
    pdf_files = []
    for root, _, files in os.walk(DATA_DIR):
        pdf_files.extend([os.path.join(root, f) for f in files if f.lower().endswith(".pdf")])

    # 使用tqdm显示进度
    for pdf_path in tqdm(pdf_files, desc="提取PDF文本"):
        extract_text_to_txt(pdf_path, txt_output_dir)

    print(f"文本提取完成，结果保存至：{txt_output_dir}")

if __name__ == "__main__":
    main()

提取PDF文本:  78%|███████▊  | 72/92 [02:00<00:06,  2.91it/s]

处理文件 ../data/20250401\弘业期货_323616.PDF 时出错: No /Root object! - Is this really a PDF?
处理文件 ../data/20250401\恒泰期货_323467.PDF 时出错: No /Root object! - Is this really a PDF?
处理文件 ../data/20250401\恒泰期货_323468.PDF 时出错: No /Root object! - Is this really a PDF?
处理文件 ../data/20250401\恒泰期货_323470.PDF 时出错: No /Root object! - Is this really a PDF?
处理文件 ../data/20250401\恒泰期货_323577.PDF 时出错: No /Root object! - Is this really a PDF?
处理文件 ../data/20250401\恒泰期货_323578.PDF 时出错: No /Root object! - Is this really a PDF?


提取PDF文本:  95%|█████████▍| 87/92 [02:03<00:01,  3.00it/s]

处理文件 ../data/20250401\迈科期货_324419.PDF 时出错: No /Root object! - Is this really a PDF?


提取PDF文本: 100%|██████████| 92/92 [02:07<00:00,  1.39s/it]

文本提取完成，结果保存至：../output\text_files


### 需要提前下载 wkhtmltopdf 的可执行文件，并配置路径
下载链接：https://wkhtmltopdf.org/downloads.html

In [ ]:
import pdfkit
import os
from pathlib import Path
from tqdm import tqdm  

# 配置wkhtmltopdf路径
path_wkhtmltopdf = r'C:\Program Files\wkhtmltopdf\bin\wkhtmltopdf.exe'
config = pdfkit.configuration(wkhtmltopdf=path_wkhtmltopdf)

for i in range(20250401,20250429):
    input_folder = f'../data/{i}/'  
    output_folder = f'../output/{i}/'
    print(f"正在转换{i}。。。")

    # 1. 递归收集所有HTML文件路径
    html_files = []
    for root, _, files in os.walk(input_folder):
        for file in files:
            if file.lower().endswith('.html'):
                html_files.append(os.path.join(root, file))

    # 2. 初始化tqdm进度条
    with tqdm(total=len(html_files), desc="转换PDF", unit="file", 
            bar_format="{l_bar}{bar}| {n_fmt}/{total_fmt} [{elapsed}<{remaining}]") as pbar:
        
        for input_path in html_files:
            # 3. 动态更新进度条描述（显示当前文件名）
            pbar.set_postfix(file=os.path.basename(input_path)[:20] + "...")
            
            # 4. 构建输出路径（保留原始目录结构）
            relative_path = os.path.relpath(os.path.dirname(input_path), input_folder)
            output_subdir = os.path.join(output_folder, relative_path)
            Path(output_subdir).mkdir(parents=True, exist_ok=True)
            pdf_filename = os.path.splitext(os.path.basename(input_path))[0] + '.pdf'
            output_path = os.path.join(output_subdir, pdf_filename)
            
            try:
                # 5. 转换文件
                pdfkit.from_file(
                    input=input_path,
                    output_path=output_path,
                    configuration=config,
                    options={
                        'encoding': 'UTF-8',
                        'enable-local-file-access': True,
                        'quiet': '' 
                    }
                )
            except OSError as e:
                if "ContentNotFoundError" in str(e):
                    # tqdm.write(f"⚠ 警告: {input_path} 资源缺失（PDF已生成）")
                    pass
                else:
                    tqdm.write(f"✗ 失败: {input_path} - {str(e)}")
            except Exception as e:
                tqdm.write(f"✗ 异常: {input_path} - {str(e)}")
            
            # 6. 更新进度
            pbar.update(1)  # 重要：每次循环后进度+1

    print(f"\n转换完成！PDF保存在: {os.path.abspath(output_folder)}")

正在转换20250408。。。


转换PDF:   6%|▌         | 13/236 [00:33<29:19]  

✗ 失败: ../data/20250408/中衍期货_325216_0.html - wkhtmltopdf reported an error:
Exit with code 1 due to network error: HostNotFoundError



转换PDF:   6%|▌         | 14/236 [00:34<22:12]  

✗ 失败: ../data/20250408/中衍期货_325217_0.html - wkhtmltopdf reported an error:
Exit with code 1 due to network error: HostNotFoundError



...

✗ 失败: ../data/20250429/中衍期货_332707_0.html - wkhtmltopdf reported an error:
Exit with code 1 due to network error: HostNotFoundError



转换PDF: 100%|██████████| 59/59 [01:38<00:00]



转换完成！PDF保存在: h:\project\economy\output\20250429
正在转换20250430。。。


转换PDF: |          | 0/0 [00:00<?]


转换完成！PDF保存在: h:\project\economy\output\20250430


In [3]:
import os
from PyPDF2 import PdfReader
from tqdm import tqdm
from openpyxl import Workbook

DATA_DIR = "../data"  # 数据文件夹路径
OUTPUT_DIR = "../output"  # 输出文件夹路径
# 记录损坏文件的 XLSX 文件路径
BAD_PDF_RECORD = os.path.join(OUTPUT_DIR, "bad_pdfs.xlsx")


def is_pdf_valid(pdf_path):
    """检查PDF文件是否损坏，并记录损坏文件信息"""
    try:
        with open(pdf_path, 'rb') as f:
            PdfReader(f)
        return True
    except Exception as e:
        print(f"文件 {pdf_path} 损坏: {str(e)}")
        return False


def copy_valid_pdfs():
    # 创建工作簿和工作表
    wb = Workbook()
    ws = wb.active
    ws.append(['文件名', '来源路径'])

    # 收集所有日期文件夹
    date_folders = []
    for root, dirs, _ in os.walk(DATA_DIR):
        for dir_name in dirs:
            if dir_name.isdigit() and len(dir_name) == 8:  # 假设日期文件夹名为 8 位数字
                date_folders.append(os.path.join(root, dir_name))

    # 遍历每个日期文件夹
    for date_folder in tqdm(date_folders, desc="处理文件夹"):
        pdf_files = []
        for root, _, files in os.walk(date_folder):
            for file in files:
                if file.lower().endswith('.pdf'):
                    pdf_files.append(os.path.join(root, file))

        # 处理当前日期文件夹中的PDF文件
        for pdf_path in tqdm(pdf_files, desc=f"处理 {os.path.basename(date_folder)}", leave=False):
            if not is_pdf_valid(pdf_path):
                # 记录损坏文件信息到 XLSX
                file_name = os.path.basename(pdf_path)
                ws.append([file_name, pdf_path])
            else:
                # 获取日期文件夹名，如 20250401
                date_folder_name = os.path.basename(date_folder)
                output_date_dir = os.path.join(OUTPUT_DIR, date_folder_name)
                os.makedirs(output_date_dir, exist_ok=True)
                output_path = os.path.join(output_date_dir, os.path.basename(pdf_path))
                # 复制文件
                with open(pdf_path, 'rb') as src, open(output_path, 'wb') as dst:
                    dst.write(src.read())

        print(f"文件夹 {date_folder_name} 处理完成")

    # 保存 XLSX 文件
    wb.save(BAD_PDF_RECORD)
    print(f"损坏的 PDF 文件信息已记录到 {BAD_PDF_RECORD}")


if __name__ == "__main__":
    copy_valid_pdfs()

处理文件夹:   3%|▎         | 1/29 [00:01<00:48,  1.72s/it]

文件 ../data\20250401\弘业期货_323616.PDF 损坏: EOF marker not found
文件 ../data\20250401\恒泰期货_323467.PDF 损坏: EOF marker not found
文件 ../data\20250401\恒泰期货_323468.PDF 损坏: EOF marker not found
文件 ../data\20250401\恒泰期货_323470.PDF 损坏: EOF marker not found
文件 ../data\20250401\恒泰期货_323577.PDF 损坏: EOF marker not found
文件 ../data\20250401\恒泰期货_323578.PDF 损坏: EOF marker not found
文件 ../data\20250401\迈科期货_324419.PDF 损坏: EOF marker not found
文件夹 20250401 处理完成


文件 ../data\20250402\恒泰期货_323751.PDF 损坏: EOF marker not found
文件 ../data\20250402\恒泰期货_323752.PDF 损坏: EOF marker not found
文件 ../data\20250402\恒泰期货_323763.PDF 损坏: EOF marker not found
文件 ../data\20250402\迈科期货_324418.PDF 损坏: EOF marker not found


处理文件夹:   7%|▋         | 2/29 [00:03<00:40,  1.49s/it]

文件夹 20250402 处理完成


...

处理文件夹:  97%|█████████▋| 28/29 [02:31<00:07,  7.26s/it]

文件夹 20250428 处理完成


文件 ../data\20250429\恒泰期货_333123.PDF 损坏: EOF marker not found
文件 ../data\20250429\恒泰期货_333124.PDF 损坏: EOF marker not found
文件 ../data\20250429\恒泰期货_333125.PDF 损坏: EOF marker not found
文件 ../data\20250429\恒泰期货_333126.PDF 损坏: EOF marker not found


处理文件夹: 100%|██████████| 29/29 [02:36<00:00,  5.40s/it]

文件夹 20250429 处理完成
损坏的 PDF 文件信息已记录到 ../output\bad_pdfs.xlsx
